In [1]:
# Imports
import json
import pandas as pd
from neo4j import GraphDatabase
import spacy # add to poetry
import re
from collections import Counter
import time
import pickle


## Neo4j-Verbindung einrichten

**Wichtig:** Es wird eine laufende Neo4j-Instanz benötigt.
- Entweder lokal installiert (neo4j.com/download)
- Oder Neo4j Desktop
- Oder Docker: `docker run --name neo4j-rag --publish=7474:7474 --publish=7687:7687 --env NEO4J_AUTH=neo4j/password neo4j:latest`

Standard-Credentials: neo4j/password

In [2]:
# Neo4j-Verbindung konfigurieren
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    # Test-Verbindung
    with driver.session() as session:
        result = session.run("RETURN 'Neo4j verbunden!' AS message")
        print(f"{result.single()['message']}")
except Exception as e:
    print(f"Neo4j-Verbindung fehlgeschlagen: {e}")
    print("Starte Neo4j-Server oder ändere die Verbindungsparameter")

Neo4j verbunden!


## Daten laden

In [3]:
# FAQ-Korpus laden
with open('../data/faq_korpus.json', 'r', encoding='utf-8') as f:
    faq_documents = json.load(f)

# Testfragen laden
test_questions = pd.read_csv('../data/fragenliste.csv')

print(f"{len(faq_documents)} FAQ-Dokumente geladen")
print(f"{len(test_questions)} Testfragen geladen")

15 FAQ-Dokumente geladen
12 Testfragen geladen


## Spacy für Named Entity Recognition laden

In [7]:
# Deutsches Spacy-Modell laden
try:
    nlp = spacy.load("de_core_news_sm")
    print("Deutsches Spacy-Modell geladen")
except OSError:
    # Fallback auf englisches Modell
    try:
        nlp = spacy.load("en_core_web_sm")
        print("Englisches Spacy-Modell geladen (Fallback)")
    except OSError:
        print("Kein Spacy-Modell gefunden. Installieren mit:")
        print("python -m spacy download de_core_news_sm")
        print("oder: python -m spacy download en_core_web_sm")

Deutsches Spacy-Modell geladen


## Entitäten und Beziehungen extrahieren

In [8]:
def extract_entities_and_concepts(text):
    """
    Extrahiert Entitäten und wichtige Konzepte aus Text
    """
    doc = nlp(text)

    # Named Entities
    entities = []
    for ent in doc.ents:
        if len(ent.text.strip()) > 2:  # Mindestlänge
            entities.append({
                'text': ent.text.strip(),
                'label': ent.label_,
                'type': 'ENTITY'
            })

    # Wichtige Noun Phrases (Konzepte)
    concepts = []
    for chunk in doc.noun_chunks:
        text = chunk.text.strip().lower()
        # Filter für relevante Konzepte
        if (len(text) > 3 and
            not text.startswith(('der', 'die', 'das', 'ein', 'eine')) and
            text not in ['frage', 'antwort', 'system', 'methode']):
            concepts.append({
                'text': text,
                'label': 'CONCEPT',
                'type': 'CONCEPT'
            })

    # Deduplizierung
    all_items = entities + concepts
    unique_items = []
    seen = set()
    for item in all_items:
        key = item['text'].lower()
        if key not in seen:
            seen.add(key)
            unique_items.append(item)

    return unique_items[:10]  # Top 10 relevante Items

# Test der Funktion
test_text = faq_documents[0]['question'] + " " + faq_documents[0]['answer']
test_entities = extract_entities_and_concepts(test_text)
print("Entity-Extraktion getestet")
print(f"Beispiel-Entitäten: {[e['text'] for e in test_entities[:5]]}")

Entity-Extraktion getestet
Beispiel-Entitäten: ['Retrieval-Augmented Generation', 'RAG', 'Large Language Models', 'Wissensquellen', '(rag']


## Knowledge Graph aufbauen

In [9]:
def clear_database():
    """Löscht alle Daten in der Neo4j-Datenbank"""
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("Datenbank geleert")

def create_graph_schema():
    """Erstellt die Graph-Schema-Constraints"""
    with driver.session() as session:
        # Constraints für eindeutige IDs
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (d:Document) REQUIRE d.id IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE e.name IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Concept) REQUIRE c.name IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (cat:Category) REQUIRE cat.name IS UNIQUE")
        print("Graph-Schema erstellt")

# Datenbank vorbereiten
clear_database()
create_graph_schema()

Datenbank geleert
Graph-Schema erstellt


In [10]:
def build_knowledge_graph():
    """Baut den Knowledge Graph aus den FAQ-Dokumenten auf"""
    print("Baue Knowledge Graph auf...")

    with driver.session() as session:
        for doc in faq_documents:
            # Dokument-Node erstellen
            session.run("""
                CREATE (d:Document {
                    id: $id,
                    question: $question,
                    answer: $answer,
                    category: $category,
                    full_text: $full_text
                })
            """, {
                'id': doc['id'],
                'question': doc['question'],
                'answer': doc['answer'],
                'category': doc['category'],
                'full_text': doc['question'] + " " + doc['answer']
            })

            # Kategorie-Node erstellen und verknüpfen
            session.run("""
                MERGE (cat:Category {name: $category})
                WITH cat
                MATCH (d:Document {id: $doc_id})
                MERGE (d)-[:BELONGS_TO]->(cat)
            """, {
                'category': doc['category'],
                'doc_id': doc['id']
            })

            # Keywords als Konzepte hinzufügen
            for keyword in doc['keywords']:
                session.run("""
                    MERGE (c:Concept {name: $keyword})
                    WITH c
                    MATCH (d:Document {id: $doc_id})
                    MERGE (d)-[:MENTIONS]->(c)
                """, {
                    'keyword': keyword.lower(),
                    'doc_id': doc['id']
                })

            # Entitäten aus Text extrahieren
            combined_text = doc['question'] + " " + doc['answer']
            entities = extract_entities_and_concepts(combined_text)

            for entity in entities:
                if entity['type'] == 'ENTITY':
                    session.run("""
                        MERGE (e:Entity {name: $name, type: $ent_type})
                        WITH e
                        MATCH (d:Document {id: $doc_id})
                        MERGE (d)-[:CONTAINS]->(e)
                    """, {
                        'name': entity['text'].lower(),
                        'ent_type': entity['label'],
                        'doc_id': doc['id']
                    })
                elif entity['type'] == 'CONCEPT':
                    session.run("""
                        MERGE (c:Concept {name: $name})
                        WITH c
                        MATCH (d:Document {id: $doc_id})
                        MERGE (d)-[:MENTIONS]->(c)
                    """, {
                        'name': entity['text'].lower(),
                        'doc_id': doc['id']
                    })

    print("Knowledge Graph aufgebaut")

# Graph aufbauen
build_knowledge_graph()

Baue Knowledge Graph auf...
Knowledge Graph aufgebaut


## Graph-Statistiken anzeigen

In [11]:
def get_graph_stats():
    """Zeigt Statistiken über den erstellten Graph"""
    with driver.session() as session:
        # Anzahl Nodes nach Typ
        stats = {}

        result = session.run("MATCH (d:Document) RETURN count(d) as count")
        stats['Documents'] = result.single()['count']

        result = session.run("MATCH (c:Concept) RETURN count(c) as count")
        stats['Concepts'] = result.single()['count']

        result = session.run("MATCH (e:Entity) RETURN count(e) as count")
        stats['Entities'] = result.single()['count']

        result = session.run("MATCH (cat:Category) RETURN count(cat) as count")
        stats['Categories'] = result.single()['count']

        # Anzahl Beziehungen
        result = session.run("MATCH ()-[r]->() RETURN count(r) as count")
        stats['Relationships'] = result.single()['count']

        return stats

# Statistiken anzeigen
stats = get_graph_stats()
print("=== KNOWLEDGE GRAPH STATISTIKEN ===")
for key, value in stats.items():
    print(f"{key}: {value}")

=== KNOWLEDGE GRAPH STATISTIKEN ===
Documents: 15
Concepts: 116
Entities: 64
Categories: 14
Relationships: 216


## Graph-basierte Retrieval-Funktion implementieren

In [12]:
def graph_retrieval(query, top_k=3):
    """
    Führt Graph-basiertes Retrieval für eine Anfrage durch
    """
    # Query-Entitäten extrahieren
    query_entities = extract_entities_and_concepts(query)
    query_terms = [e['text'].lower() for e in query_entities]

    # Auch einfache Keywords aus der Query extrahieren
    simple_terms = [word.lower().strip() for word in query.split()
                   if len(word) > 3 and word.lower() not in ['sind', 'eine', 'eines', 'wie', 'was', 'welche']]

    all_query_terms = list(set(query_terms + simple_terms))

    if not all_query_terms:
        # Fallback: Textsuche
        all_query_terms = [query.lower()]

    with driver.session() as session:
        # Cypher-Query für Graph-Traversierung
        cypher_query = """
        MATCH (d:Document)
        OPTIONAL MATCH (d)-[:MENTIONS]->(c:Concept)
        OPTIONAL MATCH (d)-[:CONTAINS]->(e:Entity)
        OPTIONAL MATCH (d)-[:BELONGS_TO]->(cat:Category)

        WITH d,
             collect(DISTINCT c.name) as concepts,
             collect(DISTINCT e.name) as entities,
             cat.name as category,
             d.full_text as full_text

        WITH d, concepts, entities, category, full_text,
             // Score berechnen basierend auf Übereinstimmungen
             REDUCE(score = 0, term IN $query_terms |
                 score +
                 CASE WHEN term IN concepts THEN 3 ELSE 0 END +
                 CASE WHEN term IN entities THEN 2 ELSE 0 END +
                 CASE WHEN toLower(d.question) CONTAINS term THEN 2 ELSE 0 END +
                 CASE WHEN toLower(d.answer) CONTAINS term THEN 1 ELSE 0 END
             ) as relevance_score

        WHERE relevance_score > 0

        RETURN d.id as doc_id,
               d.question as question,
               d.answer as answer,
               d.category as category,
               concepts,
               entities,
               relevance_score

        ORDER BY relevance_score DESC
        LIMIT $top_k
        """

        result = session.run(cypher_query, {
            'query_terms': all_query_terms,
            'top_k': top_k
        })

        # Ergebnisse formatieren
        results = []
        for i, record in enumerate(result):
            results.append({
                'rank': i + 1,
                'score': record['relevance_score'],
                'document_id': record['doc_id'],
                'question': record['question'],
                'answer': record['answer'],
                'category': record['category'],
                'matched_concepts': record['concepts'],
                'matched_entities': record['entities'],
                'query_terms': all_query_terms
            })

        return results

print("✅ Graph-Retrieval-Funktion implementiert")

✅ Graph-Retrieval-Funktion implementiert


## Erste Tests mit Graph Retrieval

In [13]:
# Test 1: Einfache Frage zu RAG
print("=== TEST 1: Einfache RAG-Frage (Graph) ===")
query1 = "Was ist RAG?"
graph_results1 = graph_retrieval(query1, top_k=3)

print(f"Query: '{query1}'")
print(f"Extrahierte Begriffe: {graph_results1[0]['query_terms'] if graph_results1 else 'Keine'}")
print("\nTop 3 Ergebnisse:")
for result in graph_results1:
    print(f"\n{result['rank']}. Score: {result['score']}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")
    print(f"   Matched Concepts: {result['matched_concepts'][:3]}")

=== TEST 1: Einfache RAG-Frage (Graph) ===
Query: 'Was ist RAG?'
Extrahierte Begriffe: ['rag?', 'rag']

Top 3 Ergebnisse:

1. Score: 8
   Frage: Was ist Retrieval-Augmented Generation (RAG)?
   Kategorie: RAG Basics
   Matched Concepts: ['aktuellere informationen', 'halluzinationen', 'seine antworten']

2. Score: 6
   Frage: Wie wählt man die richtige Chunk-Größe für RAG?
   Kategorie: Data Processing
   Matched Concepts: ['große chunks', 'präzise retrieval', '(100-200 tokens']

3. Score: 5
   Frage: Wie implementiert man RAG mit Neo4j?
   Kategorie: Graph Implementation
   Matched Concepts: ['vektorsuche', 'graphtraversierung', 'graph']


In [14]:
# Test 2: Technische Frage zu Vektoren
print("\n=== TEST 2: Technische Frage (Graph) ===")
query2 = "Wie funktioniert Vektorsuche?"
graph_results2 = graph_retrieval(query2, top_k=3)

print(f"Query: '{query2}'")
print(f"Extrahierte Begriffe: {graph_results2[0]['query_terms'] if graph_results2 else 'Keine'}")
print("\nTop 3 Ergebnisse:")
for result in graph_results2:
    print(f"\n{result['rank']}. Score: {result['score']}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")
    print(f"   Matched Concepts: {result['matched_concepts'][:3]}")


=== TEST 2: Technische Frage (Graph) ===
Query: 'Wie funktioniert Vektorsuche?'
Extrahierte Begriffe: ['vektorsuche?', 'vektorsuche', 'funktioniert']

Top 3 Ergebnisse:

1. Score: 7
   Frage: Wie funktioniert Vektorsuche in RAG-Systemen?
   Kategorie: Vector Retrieval
   Matched Concepts: ['ähnliche bibliotheken', 'andere metriken', 'cosinus-ähnlichkeit']

2. Score: 4
   Frage: Wie implementiert man RAG mit Neo4j?
   Kategorie: Graph Implementation
   Matched Concepts: ['vektorsuche', 'graphtraversierung', 'graph']

3. Score: 2
   Frage: Wie funktioniert Re-ranking in RAG-Pipelines?
   Kategorie: Advanced Techniques
   Matched Concepts: ['rag-pipelines', 'two-stage retrieval', 'cross-encoder']


In [15]:
# Test 3: Vergleichsfrage
print("\n=== TEST 3: Vergleichsfrage (Graph) ===")
query3 = "Unterschied zwischen Vektor und Graph Retrieval"
graph_results3 = graph_retrieval(query3, top_k=3)

print(f"Query: '{query3}'")
print(f"Extrahierte Begriffe: {graph_results3[0]['query_terms'] if graph_results3 else 'Keine'}")
print("\nTop 3 Ergebnisse:")
for result in graph_results3:
    print(f"\n{result['rank']}. Score: {result['score']}")
    print(f"   Frage: {result['question']}")
    print(f"   Kategorie: {result['category']}")
    print(f"   Matched Concepts: {result['matched_concepts'][:3]}")


=== TEST 3: Vergleichsfrage (Graph) ===
Query: 'Unterschied zwischen Vektor und Graph Retrieval'
Extrahierte Begriffe: ['vektor und graph retrieval', 'vektor', 'zwischen', 'retrieval', 'unterschied', 'graph']

Top 3 Ergebnisse:

1. Score: 10
   Frage: Was ist der Unterschied zwischen Dense und Sparse Retrieval?
   Kategorie: Retrieval Methods
   Matched Concepts: ['semantische suche', 'exakte keyword-matches', 'semantische ähnlichkeit']

2. Score: 8
   Frage: Wie implementiert man RAG mit Neo4j?
   Kategorie: Graph Implementation
   Matched Concepts: ['vektorsuche', 'graphtraversierung', 'graph']

3. Score: 7
   Frage: Was sind die Vorteile von Graph-basiertem Retrieval?
   Kategorie: Graph Retrieval
   Matched Concepts: ['graphtraversierung', 'kontext', 'strukturierten daten']


## Systematischer Test mit allen Testfragen

In [16]:
# Alle Testfragen mit Graph Retrieval durchlaufen
print("=== SYSTEMATISCHER TEST ALLER TESTFRAGEN (GRAPH) ===")

graph_results = []

for idx, row in test_questions.iterrows():
    query = row['question']
    question_id = row['id']
    difficulty = row['difficulty']

    # Graph-Retrieval durchführen
    results = graph_retrieval(query, top_k=3)

    # Bestes Ergebnis speichern
    if results:
        best_result = results[0]
        graph_results.append({
            'question_id': question_id,
            'query': query,
            'difficulty': difficulty,
            'retrieved_doc_id': best_result['document_id'],
            'retrieval_score': best_result['score'],
            'retrieved_question': best_result['question'],
            'retrieved_category': best_result['category'],
            'matched_concepts': len(best_result['matched_concepts']),
            'query_terms': len(best_result['query_terms'])
        })
        score = best_result['score']
    else:
        # Kein Ergebnis gefunden
        graph_results.append({
            'question_id': question_id,
            'query': query,
            'difficulty': difficulty,
            'retrieved_doc_id': 'NO_RESULT',
            'retrieval_score': 0,
            'retrieved_question': '',
            'retrieved_category': '',
            'matched_concepts': 0,
            'query_terms': 0
        })
        score = 0

    print(f"✅ {question_id}: {difficulty} - Score: {score}")

print(f"\n✅ {len(graph_results)} Testfragen abgearbeitet")

=== SYSTEMATISCHER TEST ALLER TESTFRAGEN (GRAPH) ===
✅ q001: easy - Score: 1
✅ q002: medium - Score: 2
✅ q003: medium - Score: 4
✅ q004: hard - Score: 0
✅ q005: easy - Score: 0
✅ q006: hard - Score: 11
✅ q007: medium - Score: 7
✅ q008: medium - Score: 15
✅ q009: hard - Score: 8
✅ q010: medium - Score: 0
✅ q011: easy - Score: 11
✅ q012: hard - Score: 0

✅ 12 Testfragen abgearbeitet


## Graph Retrieval Ergebnisse analysieren

In [17]:
# Ergebnisse in DataFrame konvertieren
graph_df = pd.DataFrame(graph_results)

print("=== GRAPH RETRIEVAL ANALYSE ===")
print(f"Anzahl Testfragen: {len(graph_df)}")
print(f"Erfolgreiche Retrievals: {len(graph_df[graph_df['retrieval_score'] > 0])}")
print(f"Durchschnittlicher Score: {graph_df['retrieval_score'].mean():.3f}")
print(f"Minimaler Score: {graph_df['retrieval_score'].min():.3f}")
print(f"Maximaler Score: {graph_df['retrieval_score'].max():.3f}")

# Analyse nach Schwierigkeit
print("\n=== GRAPH SCORES NACH SCHWIERIGKEIT ===")
difficulty_stats = graph_df.groupby('difficulty')['retrieval_score'].agg(['count', 'mean', 'std']).round(3)
print(difficulty_stats)

# Top 5 und Bottom 5 Ergebnisse
print("\n=== TOP 5 GRAPH RETRIEVAL ERGEBNISSE ===")
top_5 = graph_df.nlargest(5, 'retrieval_score')[['question_id', 'difficulty', 'retrieval_score', 'query']]
print(top_5.to_string(index=False))

print("\n=== SCHWÄCHSTE 5 GRAPH RETRIEVAL ERGEBNISSE ===")
bottom_5 = graph_df.nsmallest(5, 'retrieval_score')[['question_id', 'difficulty', 'retrieval_score', 'query']]
print(bottom_5.to_string(index=False))

=== GRAPH RETRIEVAL ANALYSE ===
Anzahl Testfragen: 12
Erfolgreiche Retrievals: 8
Durchschnittlicher Score: 4.917
Minimaler Score: 0.000
Maximaler Score: 15.000

=== GRAPH SCORES NACH SCHWIERIGKEIT ===
            count  mean    std
difficulty                    
easy            3  4.00  6.083
hard            4  4.75  5.620
medium          5  5.60  5.857

=== TOP 5 GRAPH RETRIEVAL ERGEBNISSE ===
question_id difficulty  retrieval_score                                                   query
       q008     medium               15 Was sind häufige Probleme bei RAG und wie löse ich sie?
       q006       hard               11                 Wie implementiere ich Hybrid Retrieval?
       q011       easy               11                  Welche Zukunftstrends gibt es bei RAG?
       q009       hard                8              Wie funktioniert RAG mit Knowledge Graphs?
       q007     medium                7               Welche Vector Database sollte ich wählen?

=== SCHWÄCHSTE 5 GRAPH RE

## Ergebnisse speichern

In [18]:
# Graph Retrieval Ergebnisse speichern
graph_df.to_csv('../results/graph_retrieval_results.csv', index=False, encoding='utf-8')

# Graph-Metadaten speichern
graph_metadata = {
    'neo4j_uri': NEO4J_URI,
    'graph_stats': get_graph_stats(),
    'creation_time': time.time()
}

with open('../data/graph_metadata.pkl', 'wb') as f:
    pickle.dump(graph_metadata, f)

print("Graph-Ergebnisse gespeichert:")
print("  - ../results/graph_retrieval_results.csv")
print("  - ../data/graph_metadata.pkl")

Graph-Ergebnisse gespeichert:
  - ../results/graph_retrieval_results.csv
  - ../data/graph_metadata.pkl


In [19]:
print("\n=== GRAPH RETRIEVAL ABGESCHLOSSEN ===")
print("Knowledge Graph in Neo4j aufgebaut")
print("12 Testfragen erfolgreich abgearbeitet")
print("Ergebnisse gespeichert")
print(f"Durchschnittlicher Retrieval-Score: {graph_df['retrieval_score'].mean():.3f}")
print(f"Erfolgreiche Retrievals: {len(graph_df[graph_df['retrieval_score'] > 0])}/12")

# Neo4j-Verbindung schließen
driver.close()
print("Neo4j-Verbindung geschlossen")


=== GRAPH RETRIEVAL ABGESCHLOSSEN ===
Knowledge Graph in Neo4j aufgebaut
12 Testfragen erfolgreich abgearbeitet
Ergebnisse gespeichert
Durchschnittlicher Retrieval-Score: 4.917
Erfolgreiche Retrievals: 8/12

Neo4j-Verbindung geschlossen
